# Test API

In [ ]:
from routers.p2_content_creator import *

In [6]:
from dataclasses import dataclass

@dataclass
class ProductCategories:
    images_uri: list

In [7]:
data = ProductCategories(
    images_uri=[
        "https://storage.googleapis.com/csm-dataset/website-search/image_00.jpg",
        "https://storage.googleapis.com/csm-dataset/website-search/image_01.jpg",
        "https://storage.googleapis.com/csm-dataset/website-search/image_02.jpg",
        "https://storage.googleapis.com/csm-dataset/website-search/image_03.jpg",
        "https://storage.googleapis.com/csm-dataset/website-search/image_04.jpg",
        "https://storage.googleapis.com/csm-dataset/website-search/image_05.jpg",
        "https://storage.googleapis.com/csm-dataset/website-search/image_06.jpg",
    ]
)

In [ ]:
test = post_product_categories(data)
test

# AsyncIO

In [9]:
import asyncio
from vertexai.vision_models import ImageCaptioningModel, Image
imagen_model = ImageCaptioningModel.from_pretrained("imagetext")

In [10]:
from dataclasses import dataclass

@dataclass
class ProductCategories:
    images_uri: list

In [11]:
data = ProductCategories(
    images_uri=[
        "https://storage.googleapis.com/csm-dataset/website-search/image_00.jpg",
        "https://storage.googleapis.com/csm-dataset/website-search/image_01.jpg",
        "https://storage.googleapis.com/csm-dataset/website-search/image_02.jpg",
        "https://storage.googleapis.com/csm-dataset/website-search/image_03.jpg",
        "https://storage.googleapis.com/csm-dataset/website-search/image_04.jpg",
        "https://storage.googleapis.com/csm-dataset/website-search/image_05.jpg",
        "https://storage.googleapis.com/csm-dataset/website-search/image_06.jpg",
    ]
)

In [12]:
async def get_image_captions(image_uri):
    captions = await imagen_model.get_captions(
        image=Image(image_bytes=requests.get(image_uri).content),
        number_of_results=1,
        language="en",
    )
    return captions

In [15]:
loop = asyncio.get_event_loop()

asyncio.run(get_image_captions(data.images_uri[0]))

RuntimeError: asyncio.run() cannot be called from a running event loop

# Populate Firestore with products

In [ ]:
import base64
from google.cloud import aiplatform
from google.cloud import firestore
from google.protobuf import struct_pb2
import typing
import numpy as np
import pandas as pd

db = firestore.Client()

In [ ]:
input_file = pd.read_excel("csm-dataset-update.xlsx")

In [ ]:
input_file

In [ ]:
update = {}

for i in input_file.iterrows():
    db.collection("website_products").document(
        i[1]["id"]).set(
            {
                "product_id":i[1]["id"],
                "link":i[1]["link"].replace("gs://", "https://storage.googleapis.com/"),
                "title":i[1]["title"],
                "categories": [i[1]["category"]]
            }
        )